In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

cap = cv2.VideoCapture(0)
kernel = np.ones((5,5),np.uint8)
selector = 0
names = ["Dilate","Erode","Open","Close","Dil+Ero","Gradient+Median"]
kerneld = np.ones((4,4),np.uint8)
kernele = np.ones((5,5),np.uint8)
kerneleg = np.ones((4,4),np.uint8)

def on_trackbar(val):
    global selector
    selector = val

cv2.namedWindow('maskd')
cv2.createTrackbar('Select','maskd',0,len(names)-1,on_trackbar)

while(1):

    # Captura un frame
    _, frame = cap.read()

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # define range of green color in HSV
    lower_green = np.array([45,80,80])
    upper_green = np.array([80,255,255])
    
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_green, upper_green)
    match selector:
        case 0:
            maskd = cv2.dilate(mask,kernel,iterations = 1) #1
        case 1:
            maskd = cv2.erode(mask,kernel,iterations = 1) #2
        case 2: 
            maskd = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel) # 3 (erotion and dilation)
        case 3:
            maskd = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel) # 4 (dilation and erotion)
        case 4:
            maskd = cv2.dilate(mask,kerneld,iterations = 1) #5
            maskd = cv2.erode(maskd,kernele,iterations = 1) #5
        case 5:
            maskd = cv2.morphologyEx(maskd, cv2.MORPH_GRADIENT, kernel) # 6
            maskd = cv2.medianBlur(mask,11)
    
    #maskd = cv2.medianBlur(mask,11)
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= maskd)
    cv2.putText(maskd,names[selector],(10,30), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
    #cv2.imshow('frame',frame)
    cv2.imshow('mask',mask)
    cv2.imshow('maskd',maskd)
    #cv2.imshow('res',res)
    k = cv2.waitKey(10) & 0xFF
    if k == 27:
        break #Press 'Esc' to close.
        
cap.release()
cv2.destroyAllWindows()